In [6]:
import pandas as pd

In [1]:
#import boston311 stuff
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree, Boston311KerasNLP


2023-10-06 15:01:09.950781: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 15:01:09.990931: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 15:01:09.991461: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 15:01:10.483793: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


In [2]:
data = Boston311LogReg(train_date_range={'start':'2010-12-31','end':'2023-12-31'}).load_data()

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:262: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:262: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:262: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:262: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:262: DtypeWarning: Columns (13) have mixed types. Specify dtype opt

Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [3]:
#save data to csv as all_311_cases.csv
data.to_csv('all_311_cases.csv', index=False)

In [4]:
#save data to a pkl file as all_311_cases.pkl
data.to_pickle('all_311_cases.pkl')

In [7]:
#load boston_311_data_2022.csv and boston_311_data.csv and boston_311_data_predict and merge them into one dataframe removi ng duplicates
data = pd.read_csv('boston_311_data_2022.csv')
data1 = pd.read_csv('boston_311_data.csv')
data2 = pd.read_csv('boston_311_data_predict.csv')
data = pd.concat([data,data1,data2],ignore_index=True)
data = data.drop_duplicates(subset=['service_request_id'])

In [8]:
#write data to a csv file as all_311_cases_api.csv
data.to_csv('all_311_cases_api.csv', index=False)

In [14]:
#load empty_response_ids.csv, remove duplicates, and save it again as empty_response_ids.csv
data = pd.read_csv('empty_response_ids.csv')
data = data.drop_duplicates(subset=['case_enquiry_id'])
data.to_csv('empty_response_ids.csv', index=False)

In [17]:
import pandas as pd
import requests
import csv
import time
import json
import os

# Load your CSVs into pandas DataFrames

case_enquiry_id_file = "all_311_cases.csv"
service_request_id_file = "all_311_cases_api.csv"
case_enquiry_id_pkl = "all_311_cases.pkl"
empty_response_file = "empty_response_ids.csv"

#check if pkl file exists
if os.path.exists(case_enquiry_id_pkl):
    print("pkl file exists")
    df1 = pd.read_pickle(case_enquiry_id_pkl)
else:
    print("pkl file does not exist")
    df1 = pd.read_csv(case_enquiry_id_file)

#file with service_request_id
df2 = pd.read_csv(service_request_id_file)

#check if empty_response_file exists
if os.path.exists(empty_response_file):
    print("empty_response_file exists")
    df3 = pd.read_csv(empty_response_file)


# Identify missing service_request_ids
missing_ids = set(df1['case_enquiry_id']) - set(df2['service_request_id'])

#also subtract the empty_response_file
if df3 is not None:
    missing_ids = missing_ids - set(df3['case_enquiry_id'])



missing_records = df1[df1['case_enquiry_id'].isin(missing_ids)]
missing_records = missing_records.sort_values(by='case_enquiry_id', ascending=False)

#initialize CSV file for empty response IDs
#if the file exists, append to it
if os.path.exists(empty_response_file):
    empty_response_csv_file = open(empty_response_file, 'a', newline='', encoding='utf-8')
    empty_response_csv_writer = csv.writer(empty_response_csv_file)
else:
    empty_response_csv_file = open(empty_response_file, 'w', newline='', encoding='utf-8')
    empty_response_csv_writer = csv.writer(empty_response_csv_file)
    empty_response_csv_writer.writerow(['case_enquiry_id'])

# Initialize CSV file
csv_file_path = service_request_id_file
csv_file = open(csv_file_path, 'a', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)

# Rate limit delay
rate_limit_delay = 6  # 6 seconds to stay within 10 requests per minute
max_exponential_backoff = 3600 

for service_request_id in missing_records['case_enquiry_id']:
    url = f"https://311.boston.gov/open311/v2/requests.json?service_request_id={service_request_id}"


    response = requests.get(url)
    
    print(f"Fetching data for service_request_id {service_request_id}")

    if response.status_code == 200:
        try:
            data = json.loads(response.text)
            if not data:  # if data is empty
                empty_response_csv_writer.writerow([service_request_id])
                empty_response_csv_file.flush()
                print(f"Empty response for service_request_id {service_request_id}")
            else:
                for record in data:
                    #print all the data
                    #print(record.get('service_request_id'), record.get('status'), record.get('service_name'), record.get('service_code'), record.get('description'), record.get('requested_datetime'), record.get('updated_datetime'), record.get('address'), record.get('lat'), record.get('long'), record.get('token'))
                    csv_writer.writerow([
                        record.get('service_request_id'),
                        record.get('status'),
                        record.get('service_name'),
                        record.get('service_code'),
                        record.get('description'),
                        record.get('requested_datetime'),
                        record.get('updated_datetime'),
                        record.get('address'),
                        record.get('lat'),
                        record.get('long'),
                        record.get('token')
                    ])
                    csv_file.flush()
                    print(f"Data written for service_request_id {service_request_id}")
                rate_limit_delay = 6
        except json.JSONDecodeError:
            print(f"Failed to decode JSON for service_request_id {service_request_id}")
            # Double the delay time after failure
            rate_limit_delay = min(rate_limit_delay * 2, max_exponential_backoff)
    else:
        print(f"Failed to fetch data for service_request_id {service_request_id}")
        # Double the delay time after failure
        rate_limit_delay = min(rate_limit_delay * 2, max_exponential_backoff)

    # Rate limiting
    time.sleep(rate_limit_delay)

# Close CSV file
csv_file.close()


pkl file exists
empty_response_file exists
Fetching data for service_request_id 101005091452
Data written for service_request_id 101005091452
Fetching data for service_request_id 101005091451
Empty response for service_request_id 101005091451
Fetching data for service_request_id 101005091449
Data written for service_request_id 101005091449
Fetching data for service_request_id 101005091446
Data written for service_request_id 101005091446
Fetching data for service_request_id 101005091444
Data written for service_request_id 101005091444
Fetching data for service_request_id 101005091443
Data written for service_request_id 101005091443
Fetching data for service_request_id 101005091440
Empty response for service_request_id 101005091440
Fetching data for service_request_id 101005091438
Data written for service_request_id 101005091438
Fetching data for service_request_id 101005091437
Empty response for service_request_id 101005091437
Fetching data for service_request_id 101005091435
Data writt

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))